<a href="https://colab.research.google.com/github/amalsalilan/IPL_Infographics_Data_Analytics_-_Data_Visualization_Infosys_Internship_Oct2024/blob/K_Akshya/06_Model_Baseline_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

In [ ]:
df=pd.read_csv('/content/all_season_details_data.csv')
df.head()

<ipython-input-2-20ecdbe3bc5d>:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/all_season_details_data.csv')


,season,match_id,match_name,home_team,away_team,current_innings,innings_id,over,ball,runs,...,bowler2_maidens,bowler2_runs,bowler2_wkts,wicket_id,wkt_batsman_name,wkt_bowler_name,wkt_batsman_runs,wkt_batsman_balls,wkt_text,isRetiredHurt
0,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
new_df = df[['season', 'match_id', 'home_team', 'away_team', 'current_innings',
                      'innings_id', 'over', 'ball', 'runs', 'wicket_id','bowler1_name',
                      'wkt_batsman_name', 'wkt_batsman_runs']].copy()
# Calculate batsman's strike rate in each match/innings
new_df['strike_rate'] = (new_df['runs'] / new_df['ball']) * 100
new_df['strike_rate'] = new_df['strike_rate'].fillna(0)

new_df['over_runs'] = new_df.groupby(['match_id', 'current_innings', 'over'])['runs'].transform('sum')
new_df['over_run_rate'] = new_df['over_runs'] / (new_df['over'] + 1.0)

new_df['cumulative_runs'] = new_df.groupby(['match_id', 'current_innings'])['runs'].cumsum()

new_df['run_rate'] = new_df['cumulative_runs'] / (new_df['over'] + 0.1)

# Assuming 'wkt_batsman_name' indicates a wicket


new_df['wickets_lost'] = new_df.groupby(['match_id', 'current_innings'])['wkt_batsman_runs']\
                                                 .transform(lambda x: (x >= 0).cumsum())

new_df['cumulative_balls'] = new_df.groupby(['match_id', 'current_innings']).cumcount() + 1
new_df['cumulative_strike_rate'] = (new_df['cumulative_runs'] / new_df['cumulative_balls']) * 100


In [ ]:
new_df['wickets_taken'] = new_df['wkt_batsman_name'].notnull().astype(int)

# Calculate cumulative wickets for each bowler in the given match and innings
new_df['bowler_wickets'] = new_df.groupby(['match_id', 'current_innings', 'bowler1_name'])['wickets_taken'].cumsum()
result_df = new_df[['match_id', 'current_innings', 'bowler1_name', 'bowler_wickets']]

In [ ]:
unique_home_teams = new_df['home_team'].unique().tolist()
home_teams_df = pd.DataFrame(unique_home_teams, columns=['Teams'])
print(home_teams_df)

    Teams
0      GT
1    PBKS
2     LSG
3     SRH
4     RCB
5     CSK
6      DC
7      RR
8     KKR
9      MI
10   KXIP
11    RPS
12     GL
13    PWI
14  Kochi


In [ ]:
team_mapping = {'GT': 0, 'PBKS': 1, 'LSG': 2, 'SRH': 3,'RCB': 4, 'CSK': 5, 'DC': 6, 'RR': 7,
                'KKR': 8, 'MI': 9, 'KXIP': 10, 'RPS': 11,'GL': 12, 'PWI': 13, 'KOCHI': 14}
new_df['home_team_encoded'] = new_df['home_team'].map(team_mapping)
new_df['away_team_encoded'] = new_df['away_team'].map(team_mapping)
new_df['current_innings'] = new_df['current_innings'].map(team_mapping)
new_df.fillna(0, inplace=True)
df_result=new_df[['home_team_encoded','away_team_encoded','current_innings']]
df_result

,home_team_encoded,away_team_encoded,current_innings
0,0.0,5.0,5.0
1,0.0,5.0,5.0
2,0.0,5.0,5.0
3,0.0,5.0,5.0
4,0.0,5.0,5.0
...,...,...,...
242544,5.0,7.0,7.0
242545,5.0,7.0,7.0
242546,5.0,7.0,7.0
242547,5.0,7.0,7.0


In [ ]:
X = new_df[['home_team_encoded','away_team_encoded','current_innings','over', 'ball', 'run_rate','wickets_lost','wickets_taken','cumulative_strike_rate']]
y = new_df['cumulative_runs']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost,wickets_taken,cumulative_strike_rate
199032,13.0,8.0,13.0,12,4,5.537190,3,0,93.055556
237646,6.0,3.0,6.0,17,1,8.596491,2,0,144.117647
10462,0.0,6.0,6.0,1,5,4.545455,1,0,100.000000
39475,7.0,4.0,7.0,8,4,8.518519,0,0,150.000000
137928,3.0,5.0,3.0,9,2,6.153846,2,0,112.000000
...,...,...,...,...,...,...,...,...,...
119879,8.0,3.0,3.0,10,6,7.029703,3,0,116.393443
103694,3.0,9.0,3.0,12,4,8.016529,1,0,138.571429
131932,10.0,4.0,4.0,6,1,8.196721,2,0,151.515152
146867,8.0,3.0,3.0,18,6,7.182320,5,0,117.117117


In [ ]:
X_test

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost,wickets_taken,cumulative_strike_rate
122566,7.0,9.0,9.0,10,5,4.455446,4,1,76.271186
147656,9.0,5.0,9.0,5,2,6.078431,0,0,119.230769
27099,0.0,3.0,0.0,1,1,0.000000,0,0,0.000000
42493,5.0,9.0,5.0,2,5,0.952381,2,0,18.181818
154359,3.0,10.0,3.0,19,1,5.863874,5,0,102.752294
...,...,...,...,...,...,...,...,...,...
19877,0.0,6.0,0.0,17,5,8.362573,3,0,138.834951
217199,5.0,6.0,5.0,14,3,9.503546,3,0,157.647059
188029,8.0,7.0,7.0,3,1,4.516129,0,0,107.692308
220380,8.0,9.0,8.0,15,1,6.092715,2,0,102.222222


In [ ]:
y_train

,cumulative_runs
199032,67
237646,147
10462,5
39475,69
137928,56
...,...
119879,71
103694,97
131932,50
146867,130


In [ ]:
y_test

,cumulative_runs
122566,45
147656,31
27099,0
42493,2
154359,112
...,...
19877,143
217199,134
188029,14
220380,92


**Train the Random Forest model**

In [ ]:
rf_model=RandomForestRegressor()

In [ ]:
rf_model.fit(X_train,y_train)

RandomForestRegressor()

In [ ]:
y_rf_simple_pred = rf_model.predict(X_test)

In [ ]:
rf_simple_mse=mean_squared_error(y_test,y_rf_simple_pred)
rf_r2=r2_score(y_test,y_rf_simple_pred)

In [ ]:
print('MSE score: ',rf_simple_mse)

MSE score:  0.08428808286951146


In [ ]:
print('R2 score: ',rf_r2)

R2 score:  0.9999662270011697
